In [1]:
%cd /Users/trancatkhanh/Downloads/GAME

/Users/trancatkhanh/Downloads/GAME


In [2]:
import numpy as np
from base.MachiKoro.env import *
#0 Array bias trong mỗi trận
#1 Array bias sau 100k trận 
#2 Lưu số trận 
#3 lưu hệ số đã dùng 
#4 Tỷ lệ thắng khi đấu 10k trận 
#5 Trigger để train 100k trận 
#6 Đếm số lần yếu liên tiếp
#7 Lưu array cho policy yếu 
#8 State của trận đấu
#9 Action của trận đấu
#10 Reward của trận đấu
#11 Số trận lưu state
#12 Trigger policy yếu

perx = [np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((1,getActionSize())),np.zeros((1,1)),np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((1,2)),np.zeros((1,2)),np.zeros((1,1)),np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((2000000,getStateSize())),np.zeros((2000000,1)),np.zeros((2000000,1)),np.zeros((1,1)),np.zeros((1,2))]
                         #0                                                      #1                   #2              #3                                                            #4              #5               #6                #7                                                               #8                                   #9               #10                       #11           #12

# Các hàm check tính chất của state


In [3]:
import numpy as np
@njit
def check_min(state, reward):
    just = np.zeros((2000000,2))
    for i in range(len(reward)):
        if reward[i] == -1:
            for j in range(i,len(reward)):
                if reward[j] != -1:
                    reward[i] = reward[j]
                    break
    # replace -1 with 0
    for i in range(len(reward)):
        if reward[i] == -1:
            reward[i] = 0
    # np.mean each array in the state and the corresponding reward
    for i in range(len(reward)):
        just[i][0] = np.mean(state[i])
        just[i][1] = (reward[i][0])
    # count the number of times each value of the state appears and the reward of each value of the state
    count = np.zeros((2000000,3))
    for i in range(len(reward)):
        for j in range(len(count)):
            if just[i][0] == count[j][0]:
                count[j][1] += 1
                count[j][2] += just[i][1]
                break
            if count[j][0] == 0:
                count[j][0] = just[i][0]
                count[j][1] += 1
                count[j][2] += just[i][1]
                break
    # find the index value the lowest rate of reward has the highest number of times appear
    min = 0
    for i in range(len(count)):
        if count[i][1] > count[min][1]:
            min = i
    for i in range(len(count)):
        if count[i][1] == count[min][1] and count[i][2] < count[min][2]:
            min = i

    # min = 10000000
    # min_index = 0
    # for i in range(len(count)):
    #     if count[i][1] != 0:
    #         if count[i][2]/count[i][1] < min:
    #             min = count[i][2]/count[i][1]
    #             min_index = i

    # print('mean',count[min][0])
    # print('frequency',count[min][1])
    # print('value',count[min][2])

    return count[min][0]
@njit
def check_weaker(min,x):
    if x == min:
        return 1
    else:
        return 0


# Agent Hoàn Chỉnh

In [4]:
@njit()
def p_bias(state,per):
    # Chuyển Mode gồm 3 mode mode 0 - train bias , mode 1 - test bias , mode 2 - train policy yếu ,mode 3 - test policy yếu,mode 4 - lưu state
    if per[4][0][0] > per[5][0][1] and per[4][0][1] == 10000 and per[5][0][0] == 1:
      per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
      # Lưu lại kết quả thắng
      per[5][0][1] = 0 
      per[5][0][1] += (per[4][0][0])
      # Rest số trận thắng và số trận đấu 
      per[4][0][0] = 0
      per[4][0][1] = 0 
      # Chuyển sang train bias
      per[5][0][0] = 0 
      per[2][0][0] = 0
      per[1][0] = np.zeros((1,getActionSize()))
      # Reset số lần yếu liên tiếp
      per[6][0][0] = 0 
    elif per[5][0][1] > per[4][0][0] and per[4][0][1] == 10000 and per[5][0][0] == 1:
      per[6][0][0] +=1 
      # Chuyển sang train bias
      per[5][0][0] = 0 
      per[2][0][0] = 0
      per[1][0] = np.zeros((1,getActionSize()))
      # Rest số trận thắng và số trận đấu 
      per[4][0][0] = 0
      per[4][0][1] = 0 

    # Bắt đầu mode lưu state
    if per[6][0][0] == 3 and per[11][0][0] < 2000000 :
      # Chuyển sang lưu state 
      per[5][0][0] = 4
      per[2][0][0] = 0
      # Rest số trận thắng và số trận đấu 
      per[4][0][1] = 0 
      per[4][0][0] = 0
      
    # Hết mode lưu state chuyển sang train policy yếu
    if per[5][0][0] == 4 and per[11][0][0] == 2000000:
      per[5][0][0] = 2

    # Bất đầu mode train và test policy yếu hơn
    if per[11][0][0] == 2000000:
      if per[4][0][0] > per[5][0][1] and per[4][0][1] == 10000 and per[5][0][0] == 3:
        per[7][0] = np.copy(per[1][0]/np.max(per[1][0]))
        # Lưu lại kết quả thắng
        per[5][0][1] = 0 
        per[5][0][1] += (per[4][0][0])
        # Rest số trận thắng và số trận đấu 
        per[4][0][0] = 0
        per[4][0][1] = 0 
        # Chuyển sang train
        per[5][0][0] = 2 
        per[2][0][0] = 0
        per[1][0] = np.zeros((1,getActionSize()))

      elif per[5][0][1] > per[4][0][0] and per[4][0][1] == 10000 and per[5][0][0] == 3:
        # Chuyển sang train
        per[5][0][0] = 2 
        per[2][0][0] = 0
        per[1][0] = np.zeros((1,getActionSize()))
        # Rest số trận thắng và số trận đấu 
        per[4][0][0] = 0
        per[4][0][1] = 0 
      
    # reset per[1][0] sau 100k trận
    if int(per[2][0][0]) == 100000 and per[11][0][0] < 2000000:
      per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
      per[2][0][0] = 0
      per[5][0][0] = 1
    # reset per[1][0] sau 10k trận
    if int(per[2][0][0]) == 100000 and per[11][0][0] == 2000000:
      per[7][0] = np.copy(per[1][0]/np.max(per[1][0]))
      per[2][0][0] = 0
      per[5][0][0] = 3
      
    if per[11][0][0] == 2000000 and per[12][0][0] == 0:
      per[12][0][0] += check_min(per[8], per[10])

    # Chế độ train bias
    if int(per[2][0][0]) < 100000 and per[5][0][0] == 0  :
      actions = getValidActions(state)
      actions *= per[0][0]
      action = np.argmax(actions)
      if getReward(state) == 1 :
          per[1] += per[0][0]
      if getReward(state) == 0:
          per[0][0] = np.random.rand(getActionSize())

    # Chế độ test bias
    elif int(per[2][0][0]) <10000  and per[5][0][0] == 1 : 
      list_action2 = getValidActions(state)
      action = np.argmax(list_action2*per[3][0])    
      if getReward(state) == 1:
        per[4][0][0]+=1
      if getReward(state) != -1:
        per[4][0][1]+=1  

    # Chế độ train policy yếu
    elif int(per[2][0][0]) < 100000 and per[5][0][0] == 2 and per[11][0][0] == 2000000 :
      actions = getValidActions(state)
      # Check xem policy có yếu không 
      Check_yeu = check_weaker(per[12][0][0],np.mean(state))
      if Check_yeu == 1:
        per[12][0][1] = Check_yeu
        actions *= per[0][0]
        action = np.argmax(actions)
      else:
        actions *= per[3][0]
        action = np.argmax(actions)
      if getReward(state) == 1 and per[12][0][1] == 1:
        per[1] += per[0][0]
        per[12][0][1] = 0 
      if getReward(state) == 0 and per[12][0][1] == 1:
        per[0][0] = np.random.rand(getActionSize())
        per[12][0][1] = 0 

    # chế độ test policy yếu
    elif int(per[2][0][0]) <10000 and  per[5][0][0] == 3 and per[11][0][0] == 2000000:
      actions = getValidActions(state)
      # Check xem policy có yếu không 
      if check_weaker(per[12][0][0],np.mean(state)) == 1:
        actions *= per[7][0]
        action = np.argmax(actions)
      else:
        actions *= per[3][0]
        action = np.argmax(actions)
      if getReward(state) == 1:
        per[4][0][0]+=1
      if getReward(state) != -1:
        per[4][0][1]+=1  
    

    # Báo chuyển sang chế độ lưu state và action
    if per[5][0][0] == 4 and per[11][0][0] < 2000000:
      list_action2 = getValidActions(state)
      action = np.argmax(list_action2*per[3][0]) 


    # Lưu state và action
    if getReward(state) != -2 and per[11][0][0] < 2000000 and per[5][0][0] == 4:
        for i in range(getStateSize()):
          per[8][int(per[11][0][0])][i] += state[i]        
        per[9][int(per[11][0][0])][0] += int(action)
        per[10][int(per[11][0][0])][0] += int(getReward(state))
        per[11][0][0] += 1

    # đếm số ván đấu 
    if getReward(state) != -1 :
        per[2][0][0] += 1
    return action,per

win, bias = numba_main_2(p_bias, 1000000,perx,0)

In [5]:
perx

[array([[0.34866704, 0.893254  , 0.98351933, 0.26895493, 0.9836634 ,
         0.5383364 , 0.2285094 , 0.86378655, 0.32642125, 0.44119448,
         0.02450083, 0.50313488, 0.47965049, 0.55468383, 0.81929328,
         0.53177996, 0.70011635, 0.61695875, 0.42161354, 0.86726886,
         0.90087022, 0.71884752, 0.42889055, 0.84437498, 0.44244691,
         0.12102757, 0.78544474, 0.33020078, 0.94151351, 0.45947932,
         0.4729619 , 0.22669216, 0.44591099, 0.68368444, 0.6742102 ,
         0.81296599, 0.62866744, 0.56224217, 0.43343148, 0.68502074,
         0.52157724, 0.77397584, 0.31311059, 0.12221794, 0.03756021,
         0.43717766, 0.86328844, 0.67574153, 0.02732229, 0.74158119,
         0.65705728, 0.29419485, 0.17198422, 0.1218909 ]]),
 array([[31465.86599771, 31897.70452476, 30799.47834446, 31684.05099116,
         31260.29868885, 31306.33865428, 30669.65057636, 31389.392865  ,
         31350.66673037, 31380.66100122, 30779.31862649, 31079.71095547,
         30813.78203994, 31029.

In [6]:
check_min(perx[8],perx[10])

0.9629629629629629

# Test lại tỷ lệ thắng của level 1 game MachiKoro

**Khi chưa khắc phục policy yếu :24551** 


**Khi đã khắc phục policy yếu :27961**

In [7]:
@njit()
def p_bias_1(state,per):
    actions = getValidActions(state)
    # Check xem policy có yếu không 
    if check_weaker(per[12][0][0],np.mean(state)) == 1:
      actions *= per[7][0]
      action = np.argmax(actions)
    else:
      actions *= per[3][0]
      action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(p_bias_1, 100000,perx,1)
print(win)

27236
